<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Notebooks/NB19_02_Anomaly%20Detection%20With%20Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anomaly Detection with Autoencoders

* Quando você vê um gato, você sabe que é um gato. Na terminologia da Rede Neural Artificial, é como se nossos cérebros tivessem sido treinados inúmeras vezes para distinguir um gato de um cachorro. Inspirada nas redes de um cérebro, uma RNA tem muitas camadas e neurônios com unidades de processamento simples. Um modelo de RNA treina nas imagens de gatos e cães (o valor de entrada X) e no rótulo/target “gato” e “cachorro” (o valor alvo Y). Assim, ele pode prever o “gato” (o valor Y) quando dada a imagem de um gato (os valores X).
* Um **autoencoder é um tipo especial de rede neural que copia os valores de entrada para os valores de saída**. Não requer a variável de destino como o Y convencional, portanto, é categorizado como **aprendizado NÃO supervisionado**.
* Em **Autoencoder, o número de neurônios das camadas de entrada e saída corresponde ao número de variáveis**, e o **número de neurônios das camadas ocultas é sempre menor do que o das camadas externas**.
* **Não estamos muito interessados na camada de saída. Estamos interessados na camada escondida**.
    * Se o número de neurônios nas camadas escondidas **for menor** que o número de neurônios das camadas de entrada --> as camadas escondidas extrairão as informações essenciais dos valores de entrada. 
    * Esta condição **força as camadas escondidas a aprender a maioria dos padrões dos dados e ignorar os “ruídos”**. 
        * Portanto, em um modelo de autoencoder, **as camadas escondidas devem ter menos dimensões do que as das camadas de entrada ou saída**. 
        * Se o número de neurônios nas camadas escondidas for maior que o das camadas de entrada, a rede neural terá capacidade demais para aprender os dados. Em um caso extremo, ele poderia simplesmente copiar a entrada para os valores de saída, incluindo ruídos, sem extrair nenhuma informação essencial.

### Referências
* https://towardsdatascience.com/anomaly-detection-with-pyod-b523fc47db9


### Identificar outliers (Anomaly Detection)
* Uma vez que os padrões principais são identificados, os outliers são revelados;
* Curiosamente, durante **o processo de redução da dimensionalidade, outliers são identificados. Podemos dizer que a detecção de outliers é um subproduto da redução de dimensão**.

## Porque autoencoders?
* Já existem muitas ferramentas úteis, como Análise de Componentes Principais (PCA) para detectar outliers, por que precisamos dos autoencoders? Lembre-se de que o PCA usa álgebra linear para transformar;
* Em contraste, as técnicas de autoencoder podem realizar transformações não lineares com sua função de ativação não linear e múltiplas camadas. É mais eficiente treinar várias camadas com um autoencoder, em vez de treinar uma grande transformação com PCA. As técnicas de autoencoder, portanto, mostram seus méritos quando os problemas de dados são complexos e não lineares por natureza.

### Exemplo
* Vamos gerar 25 variáveis, 500 observações e dez por cento de outliers.

In [ ]:
!pip install pyod

### Carregar as principais libraries

In [ ]:
import numpy as np
import pandas as pd
from pyod.models.knn import KNN
from pyod.models.auto_encoder import AutoEncoder
from pyod.utils.data import generate_data

In [ ]:
perc_outliers = 0.1  # percentual de outliers que queremos gerar
n_treinamento = 500  # número de pontos/observações para treinamento que serão geradas
n_teste = 500  # número de pontos/observações para teste que serão geradas
n_features = 25 # número de features

X_treinamento, y_treinamento, X_teste, y_teste = generate_data(n_train = n_treinamento, 
                                                               n_test = n_teste, 
                                                               n_features = n_features, 
                                                               contamination = perc_outliers,                                                               
                                                               random_state = 20111974)

X_treinamento = pd.DataFrame(X_treinamento)
X_teste = pd.DataFrame(X_teste)

In [ ]:
X_treinamento.head()

**É boa prática sempre padronizar as preditoras**.

In [ ]:
# X_treinamento antes da transformação StandardScaler():
X_treinamento.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

In [ ]:
X_treinamento = ss.fit_transform(X_treinamento)
X_treinamento = pd.DataFrame(X_treinamento)
X_treinamento.describe()

In [ ]:
X_teste = ss.fit_transform(X_teste)
X_teste = pd.DataFrame(X_teste)
X_teste.describe()

Para lhe dar uma boa noção de como os dados se parecem, vamos usar PCA para mostrar os dados usando 2 componentes principais:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(2) # Número de componentes principais

X_pca = pca.fit_transform(X_treinamento)
X_pca = pd.DataFrame(X_pca)
X_pca.columns=['PC1','PC2']

In [ ]:
# Gráfico
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.scatter(X_treinamento[0], X_treinamento[1], c = y_treinamento, alpha = 0.8)
plt.title('Scatter plot')
plt.xlabel('X')
plt.ylabel('y')
plt.show()

**Comentários**: Os pontos roxos agrupados são as observações “normais” e os pontos amarelos são os outliers.

### Especificação do modelo
* A seguir, 3 sugestões de modelos:
    * **Modelo 1**: [$N_{I} = 25, N_{H_{1}} = 2, N_{H_{2}} = 2, N_{O}= 25$] --> $N_{I} = N_{O}$
    * **Modelo 2**: [$N_{I} = 25, N_{H_{1}} = 10, N_{H_{2}} = 2, N_{H_{3}} = 10, N_{O}= 25$]  --> $N_{I} = N_{O}$;
    * **Modelo 3**: [$N_{I} = 25, N_{H_{1}} = 15, N_{H_{2}} = 10, N_{H_{3}} = 2, N_{H_{4}} = 10, N_{H_{5}} = 15, N_{O}= 25$]  --> $N_{I} = N_{O}$

### Steps para construção dos modelos:
* Step 1 — Construir o modelo
* Step 2 — Determinar o ponto de corte
* Step 3 — Estatísticas e resumo para cada cluster

## Modelo 1: [$N_{I} = 25, N_{H_{1}} = 2, N_{H_{2}} = 2, N_{O}= 25$]

### Step 1: Construir o modelo

In [ ]:
ml_modelo1 = AutoEncoder(hidden_neurons = [25, 4, 4, 25], epochs = 100)
ml_modelo1.fit(X_treinamento)

### Step 2: Determinar o ponto de corte
* Vamos aplicar o modelo treinado ml_modelo1 para prever o score de anomalia para cada observação nos dados de teste. 

* Como definimos/identificamos um outlier? 
    * Um outlier é um ponto que está distante de outros pontos. Então o score de anomalia é definida pela distância. A função PyOD.decision_function() calcula a distância ou o score de anomalia para cada ponto dos dados.

In [ ]:
y_treinamento_scores = ml_modelo1.decision_scores_  
y_treinamento_scores[:50]

In [ ]:
# Estimar o score de anomalia:
y_teste_scores = ml_modelo1.decision_function(X_teste)  # score dos outliers
y_teste_scores = pd.Series(y_teste_scores)

# Gráfico
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.hist(y_teste_scores, bins = 'auto')  
plt.title("Histograma para o modelo ml_modelo1 - Anomaly Scores")
plt.show()

No histograma acima, veremos que os scores altos correspondem à frequência baixa - a evidência de outliers. Portanto, definimos o ponto de corte = 2.5 --> qualquer observação com score de anomalia > 2.5 será considerado um outlier.

### Step 3: Estatísticas e resumo para cada cluster
* Vamos atribuir essas observações com score de anomalia < 2.5 ao cluster 'Normal' e ao cluster 'Outlier' para aqueles acima de 2.5.

In [ ]:
outlier_cut_point = 2.5

In [ ]:
df_teste = X_teste.copy()
df_teste['score'] = y_teste_scores
df_teste['cluster'] = np.where(df_teste['score'] < outlier_cut_point, 'Normal', 'Outlier')
df_teste['cluster'].value_counts()

df_teste.groupby('cluster').mean()

* Quantos outliers foram identificados?

In [ ]:
df_teste['cluster'].value_counts()

O output acima mostra os valores médios das variáveis em cada cluster. Os valores do cluster ‘1’ (o cluster anormal) são bastante diferentes daqueles do cluster ‘0’ (o cluster normal). Os scorees de anomalia mostram a distância média dessas observações às outras. Um score alto significa que a observação está longe do "normal".

Observe acima que o modelo ml_modelo3 identifica 50 outliers.

## Modelo 2: [$N_{I} = 25, N_{H_{1}} = 10, N_{H_{2}} = 2, N_{H_{3}} = 10, N_{O}= 25$]

### Step 1&2: Construir o modelo & Determinar o ponto de corte

In [ ]:
ml_modelo2 = AutoEncoder(hidden_neurons = [25, 10, 2, 10, 25])
ml_modelo2.fit(X_treinamento)

In [ ]:
# Estimar os scores de anomalia
y_teste_scores = ml_modelo2.decision_function(X_teste)  
y_teste_scores = pd.Series(y_teste_scores)
y_teste_scores[:10]

In [ ]:
# Histograma
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.hist(y_teste_scores, bins = 'auto')  
plt.title("Histograma para o modelo ml_modelo2 - Anomaly Scores")
plt.show()

Novamente, vamos usar um histograma para contar a frequência pdo score de anomalia. Novamente, o ponto de corte será 2.5. Desta forma, qualquer score de anomalia > 2.5 será considerado outlier.

### Step 3: Estatísticas e resumo para cada cluster

In [ ]:
outlier_cut_point = 2.5

In [ ]:
df_teste = X_teste.copy()
df_teste['score'] = y_teste_scores
df_teste['cluster'] = np.where(df_teste['score'] < outlier_cut_point, 'Normal', 'Outlier')
df_teste['cluster'].value_counts()
df_teste.groupby('cluster').mean()

A estatística resumida do Cluster '1' (o cluster anormal) é diferente das do Cluster '0' (o cluster normal). As observações no cluster 'Outlier' são outliers.

### Outliers identificados

In [ ]:
df_teste['cluster'].value_counts()

Observe acima que o modelo ml_modelo2 identifica 50 outliers.

## Modelo 3: [$N_{I} = 25, N_{H_{1}} = 15, N_{H_{2}} = 10, N_{H_{3}} = 2, N_{H_{4}} = 10, N_{H_{5}} = 15, N_{O}= 25$]
* Steps 1, 2 & 3

### Step 1: Construir o modelo

In [ ]:
ml_modelo3 = AutoEncoder(hidden_neurons = [25, 15, 10, 2, 10,15, 25])
ml_modelo3.fit(X_treinamento)

# Estimar o score de anomalia:
y_teste_scores = ml_modelo3.decision_function(X_teste)  
y_teste_scores = pd.Series(y_teste_scores)

### Step 2: Determinar o ponto de corte

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.hist(y_teste_scores, bins='auto')  
plt.title("Histograma para o modelo ml_modelo3 - Anomaly Scores")
plt.show()

In [ ]:
outlier_cut_point = 2.5

In [ ]:
df_teste = X_teste.copy()
df_teste['score'] = y_teste_scores
df_teste['cluster'] = np.where(df_teste['score'] < outlier_cut_point, 'Normal', 'Outlier')
df_teste['cluster'].value_counts()
df_teste.groupby('cluster').mean()

Observe acima que o modelo ml_modelo3 identifica 50 outliers.

### Step 3: Estatísticas e resumo para cada cluster

In [ ]:
df_teste.groupby('cluster').mean()

Como no Módulo 1 e 2, a estatística resumida do Cluster '1' (o cluster anormal) é diferente das do Cluster '0' (o cluster normal). As observações no cluster 'Outlier' são outliers.

## Estabilidade do Modelo final
* Desenvolvemos 3 modelos para detectar outliers (anomaly detection). Mas porque desenvolvemos 3 modelos?

**Resposta**: Sabemos que as **técnicas de aprendizagem NÃO supervisionadas são poderosas na detecção de outliers. No entanto, estão sujeitas a overfitting e resultados instáveis**. A solução é treinar vários modelos e, em seguida, agregar os scores. No processo de agregação, você ainda seguirá as etapas 2 e 3 como antes. Existem quatro métodos para agregar o resultado conforme abaixo:
* Maximum of Maximum (MOM)
* Average of Maximum (AOM)
* Maximum of Average (MOA)

Primeiro, colocarei todas as previsões dos três modelos acima em um quadro de dados.

In [ ]:
from pyod.models.combination import aom, moa, average, maximization

# Armazena todas as predições em dataframes:
treinamento_scores = pd.DataFrame({'ml_modelo1': ml_modelo1.decision_scores_,
                             'ml_modelo2': ml_modelo2.decision_scores_,
                             'ml_modelo3': ml_modelo3.decision_scores_
                            })

In [ ]:
teste_scores  = pd.DataFrame({'ml_modelo1': ml_modelo1.decision_function(X_teste),
                             'ml_modelo2': ml_modelo2.decision_function(X_teste),
                             'ml_modelo3': ml_modelo3.decision_function(X_teste) 
                            })

Ao agregar as pontuações, você precisa padronizar as pontuações de diferentes modelos. Não fizemos a padronização antes? Lembre-se de que a padronização antes era padronizar as variáveis de entrada. Aqui está a padronização para as pontuações de saída.

In [ ]:
treinamento_scores_norm = ss.fit_transform(treinamento_scores)
teste_scores_norm = ss.fit_transform(teste_scores)

### Modelo - Average Method

### Step 2 — Determinar o ponto de corte
* A função average () calcula a média dos scores de outliers de vários modelos.

In [ ]:
# Combination by average
y_AOM = average(teste_scores_norm)
             
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.hist(y_AOM, bins='auto')
plt.title("Combination by average")
plt.show()

Parece que podemos identificar outlires quando anomaly score > 0. Nosso exemplo identifica 50 outliers (não mostrados).

In [ ]:
outlier_cut_point = 0

In [ ]:
df_teste = pd.DataFrame(X_teste)
df_teste['y_AOM_score'] = y_AOM
df_teste['y_AOM_cluster'] = np.where(df_teste['y_AOM_score'] < outlier_cut_point, 'Normal', 'Outlier')
df_teste['y_AOM_cluster'].value_counts()

### Step 3 — Estatísticas e resumo para cada cluster
* O código a seguir e os resultados mostram que as estatísticas resumidas do Cluster '1' (o cluster anormal) são diferentes daquelas do Cluster '0' (o cluster normal). As observações no cluster 'Outlier' são outliers.

In [ ]:
df_teste.groupby('y_AOM_cluster').mean()

### Modelo - Maximum of Maximum Method

#### Step 2 — Determinar o ponto de corte
Obtenha as pontuações atípicas de vários modelos calculando o máximo.

In [ ]:
y_MOM = maximization(teste_scores_norm)
             
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 10))
plt.hist(y_MOM, bins = 'auto')
plt.title("Combination by max")
plt.show()

Da mesma forma, parece que podemos identificar aqueles> = 0,0 como outliers. Existem 50 outliers (não mostrados).

In [ ]:
outlier_cut_point = 0

In [ ]:
df_teste = pd.DataFrame(X_teste)
df_teste['y_MOM_score'] = y_MOM
df_teste['y_MOM_cluster'] = np.where(df_teste['y_MOM_score'] < outlier_cut_point, 'Normal', 'Outlier')
df_teste['y_MOM_cluster'].value_counts()

### Step 3 — Estatísticas e resumo para cada cluster

In [ ]:
df_teste.groupby('y_MOM_cluster').mean()

### Conclusões
* O procedimento para aplicar os algoritmos parece muito viável, não é? Mais uma vez, deixe-me lembrá-lo de que variáveis criteriosas e cuidadosamente elaboradas são a base para o sucesso de um modelo de detecção de anomalias. Muitas aplicações industriais requerem engenharia de recursos complexos.

## Exercício
1. Analisar o dataframe creditcard.csv.